<a href="https://colab.research.google.com/github/CelaPablo/SOA-EA2/blob/master/HPC/Cela_Pablo_ejercicio_2_reentrega_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

En el siguiente ejercicio, se realiza la suma de 3 matrices[1] cuadradas de N cantidad de elementos ingresado por parametro.

Para realizar dicha operación, se suma cada elemento de cada matriz y se lo guarda en una nueva matriz. 

<center>R[Xi][Yi] = A[Xi][Yi] + B[Xi][Yi] + C[Xi][Yi]</center>

Con este ejercicio, se pretende entender el funcionamiento basico del Lenguaje Python [2], Google Colab [3,4] y tratamiento de imagenes a bajo nivel.

---
# Armado del ambiente

Instala en el cuaderno el módulo CUDA de Python.



In [ ]:
!pip install pycuda

In [ ]:
#@title ## Parámetros de ejecución

#@markdown ---
#@markdown ### Especifique la cantidad de elementos que conforman la fila / columna:
cantidad = 10000 #@param {type:"slider", min:1000, max:10000, step:1}


# Desarrollo - Ejecución CPU - CPUGPU.

Suma de matrices cuadradas.


In [ ]:
# ------------------------------------------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos --------------
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# ------------------------------------------------------------------------------

try:
  %matplotlib inline
  from datetime import datetime
  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from pycuda.compiler import SourceModule

  tiempo_total = datetime.now()

  matrizA = numpy.random.randint(low=0,high=50,size=(cantidad, cantidad))
  matrizB = numpy.random.randint(low=0,high=50,size=(cantidad, cantidad))
  matrizC = numpy.random.randint(low=0,high=50,size=(cantidad, cantidad))

  matrizCPU = numpy.zeros_like(matrizA)
  matrizGPU = numpy.zeros_like(matrizA)

  # Desarrollo secuencial. -----------------------------------------------------
  tiempo_secuencial = datetime.now()

  for x in range(0, cantidad):
    for y in range(0, cantidad):
      matrizCPU[x][y] = matrizA[x][y] + matrizB[x][y] + matrizC[x][y]

  tiempo_secuencial = datetime.now() - tiempo_secuencial

  # Cargo las matrices en GPU. -------------------------------------------------
  matrizAGPU = cuda.mem_alloc(matrizA.nbytes)
  matrizBGPU = cuda.mem_alloc(matrizB.nbytes)
  matrizCGPU = cuda.mem_alloc(matrizC.nbytes)
  matrizRGPU = cuda.mem_alloc(matrizGPU.nbytes)

  cuda.memcpy_htod(matrizAGPU, matrizA)
  cuda.memcpy_htod(matrizBGPU, matrizB)
  cuda.memcpy_htod(matrizCGPU, matrizC)
  cuda.memcpy_htod(matrizRGPU, matrizGPU)

  # CPU - Defino la función kernel que ejecutará en GPU ------------------------
  module = SourceModule("""
  __global__ void kernel_suma(int cant, int *Agpu, int *Bgpu, int *Cgpu, int *Rgpu)
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    int idy = threadIdx.y + blockIdx.y*blockDim.y;
    int cantidad = cant * cant;
    
    if(idx < cantidad && idy < cantidad)
    {
      int index = idx * cant + idy;
      Rgpu[index] = Agpu[index] + Bgpu[index] + Cgpu[index];
    }
  }
  """) 

 # CPU - Genero la función kernel. --------------------------------------------
  kernel = module.get_function("kernel_suma")

  tiempo_paralelo = datetime.now()

  # Se calculan las dimensiones de trabajo. ------------------------------------
  dim_hilo_x = 16
  dim_bloque_x = numpy.int((cantidad+dim_hilo_x-1) / dim_hilo_x)

  dim_hilo_y = 16
  dim_bloque_y = numpy.int((cantidad+dim_hilo_y-1) / dim_hilo_y)

  kernel(numpy.int32(cantidad), matrizAGPU, matrizBGPU, matrizCGPU, matrizRGPU, block=(dim_hilo_x, dim_hilo_y, 1), grid=(dim_bloque_x, dim_bloque_y,1))

  # GPU - Copio el resultado desde la memoria GPU. -----------------------------
  cuda.memcpy_dtoh(matrizGPU, matrizRGPU)

  tiempo_paralelo = datetime.now() - tiempo_paralelo
  tiempo_total = datetime.now() - tiempo_total

  print( "Tiempo en ejecucion secuencial:", tiempo_en_ms(tiempo_secuencial), "[ms]")
  print( "Tiempo en ejecucion paralela:", tiempo_en_ms(tiempo_paralelo), "[ms]")
  print( "Tiempo Total:", tiempo_en_ms(tiempo_total), "[ms]")

except ValueError as valerr:
  print(valerr)
except FileNotFoundError:
  print("No se pudo abrir la imagen: ", imagepath)
except ModuleNotFoundError:
  print("Primero deben instalarse las dependencias - Armado del ambiente e instalaciónn de CUDA.")
except: 
  print("Houston we have a problem!")


Tiempo en ejecucion secuencial: 143829.274 [ms]
Tiempo en ejecucion paralela: 195.0 [ms]
Tiempo Total: 147738.312 [ms]


---
# Tabla de pasos de ejecución del programa


 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura de la cantidad de valores de las filas y columnas de las matrices (N).
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random          | Crea las tres matrices de enteros con numeros aletorios - Matrices de NxN.
CPU      |  numpy.zeros_like      | Crea las matrices resultantes inicializados en 0.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  for...for...          | Se realiza la suma de las matrices en forma secuencial.
CPU      |  datetime.now()        | Toma el tiempo actual.
**GPU**  |  cuda.mem_alloc        | Reservo memoria enn GPU.
**GPU**  |  cuda.memcpy_htod      | Se copia la memoria CPU en GPU.
CPU      |  SourceModule          | Defino el codigo que va a ejecutar el kernel.
CPU      |  module.get_function() | Genera la función del kernel GPU.
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU a CPU.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  print()               | Informe de los resultados.


---
# Conclusión

Si bien el ejercicio realizado no presenta una gran complejidad, me sirvio para aprender a manipular matrices de una manera distinta a la que estoy acostumbrado (tratandolo como si fuese un array). Tambien me sirvio para entender un poco mas las funciones que ofrece Numpy y lo facil que se pueden crear y manipular arrays/matrices.

En cuanto a los resultados, la resolución en paralelo es despreciable con respecto a la ejecución secuencial. Solo se ve afectada cuando el N ingresado es demasiado chico (menor a 10). A medida que se incrementa el N ingresado, la ejecución paralela se mantiene lineal mientras que la ejecución secuencial se comporta de forma exponencial (respecto al tiempo). 

Cabe mencionar, que si el N es demasiado grande ( mayor al millon), la RAM que ofrece Colab, no nos es suficiente para realizar el calculo de la suma.

## Pasos mas relevantes
1- Generar matrices aleatorias.

2- Reservar memoria en GPU (cuda.mem_alloc).

3- Copiar datos en memoria GPU (cuda.memcpy_htod).

4- Definir la función que va a ejecutar el Kernel.

5- Calcular las dimensiones.

6- Ejecucion Paralela.


---
# Bibliografía

[1] Suma de Matrices: [WEB](https://economipedia.com/definiciones/suma-de-matrices.html)

[2] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[3] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[4] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)



